In [6]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

In [405]:
import random
from gym import Env as OpenAIEnv
from gym import spaces
import numpy as np
import pandas as pd

MAX_INT = 2147483647
MAX_STEPS = 10000

class Actions:
    Buy = 0
    Sell = 1
    Hold = 2
    N = 3

class SmartBrokerEnv(OpenAIEnv):
    def __init__(
        self,
        df_info,
        portfolio,
        obs_shape=(30, 1),
        n_actions=Actions.N,
        data_dir='../data',
    ):
        self.df_info = df_info
        self.action_space = spaces.Box(
            low=np.array([0, 0]),
            high=np.array([n_actions, 1]),
            dtype=np.uint8,
        )
        self.observation_space = spaces.Box(
            low=0,
            high=1,
            shape=obs_shape,
            dtype=np.uint8,
        )
        self.obs_shape = obs_shape
        self.df_info = df_info
        self.portfolio = portfolio
        self.n_actions = n_actions
        self.curr_step = 0
        self.data_dir = data_dir
        self._init_portfolio(load_df=True)
        
    def _init_portfolio(self, load_df=False):
        self.source = self.portfolio.get('source', 'Bitstamp')
        self.init_balance = self.portfolio.get('init_balance', 100)
        self.balance = self.init_balance
        self.entity = self.portfolio.get('entity', 'XRP')
        self.year = self.portfolio.get('year', '2021')
        self.market = self.portfolio.get('market', 'USD')
        self.duration_typ = self.portfolio.get('duration_typ', 'minute')
        self.price_typ = self.portfolio.get('price_typ', 'close')
        self.units_held = 0
        self.net_worth = self.balance
        
        if load_df:
            file = f'{self.source}_{self.entity}{self.market}_{self.year}_{self.duration_typ}.csv'
            df = pd.read_csv(f'{self.data_dir}/{file}', skiprows=1, parse_dates=True)
            df = df[self.df_info.get('cols')]
            # process and initialise dataframe
            self._process_df(df)
        
    def _process_df(self, df):
        start_dt = self.df_info.get('start_date')
        end_dt = self.df_info.get('end_date')
        norm_cols = self.df_info.get('norm_cols')
    
        # filter based on range
        self.df = df.loc[(df['date'] > start_dt) & (df['date'] <= end_dt)]
        self.df.reset_index()
        self.df[norm_cols] = self.df[norm_cols].apply(
            lambda x: (x - x.min()) / (x.max() - x.min()),
        )
        
        
    def _get_features(self):
        # normalise features
        return np.array([self.balance/MAX_INT, self.units_held/MAX_INT, self.net_worth/MAX_INT])
                          
    def _get_obs(self):
        prices = self.df.loc[
            self.curr_step: self.curr_step + self.obs_shape[0],
            self.price_typ,
        ].values 

        obs = np.append(
            prices,
            self._get_features()
        )
        
        return obs
            
    def _act(self, action):
        action_type = action[0]
        amount = action[1]
        curr_price = self.df.iloc[self.curr_step][self.price_typ]
        units_bought = 0
        units_sold = 0
        
        if action_type == Actions.Buy:
            total_possible = int(self.balance / curr_price)
            units_bought = int(total_possible * amount)
            cost = units_bought * curr_price
            self.balance -= cost
            self.units_held += units_bought
        elif action_type == Actions.Sell:
            units_sold = self.units_held * amount
            self.balance += units_sold * curr_price
            self.units_held -= units_sold
            
        self.net_worth = self.balance + self.units_held * curr_price
        
        info = {
            'amount': action[1],
            'curr_step': self.curr_step,
            'units_bought': units_bought,
            'units_sold': units_sold,
            'balance': self.balance,
            'net_worth': self.net_worth,
            'units_held': self.units_held,
        }
        
        return info
        
    def reset(self):
        self._init_portfolio()
        self.curr_step = random.randint(0, self.df.shape[0] - self.obs_shape[0])
        obs = self._get_obs()
        return obs
    
    def step(self, action):
        info = self._act(action)
        self.curr_step += 1
        
        if self.curr_step > self.df.shape[0] - self.obs_shape[0]:
            self.curr_step = random.randint(0, self.df.shape[0] - self.obs_shape[0])
            
        alpha = (self.curr_step / MAX_STEPS)
        reward = self.balance * alpha
        done = self.net_worth <= 0
        obs = self._get_obs()
        
        return obs, reward, done, info
    
    def render(self, show=False):
        print(f'curr_step: {self.curr_step}')
        print(f'balance: {self.balance}')
        print(f'net_worth: {self.net_worth}')
        print(f'units_held: {self.units_held}')
        print(f'net_profit: {self.net_worth - self.init_balance}')

In [527]:
norm_cols = ['open', 'high', 'low', 'close', f'Volume XRP']
cols = ['date'] + norm_cols
env = SmartBrokerEnv(
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-05-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={
    
    },
)

In [530]:
env.step([Actions.Buy, 1])

1
balance=99.4370667077492, total_possible=126, curr_price=0.7871183462001516, cost=99.1769116212191


(array([1.21144153e-10, 5.86733222e-08, 4.63039925e-08]),
 7.804652595902723e-05,
 False,
 {'amount': 1,
  'curr_step': 2,
  'units_bought': 126,
  'units_sold': 0,
  'balance': 0.2601550865300908,
  'net_worth': 99.4370667077492,
  'units_held': 126})

In [531]:
env.step([Actions.Sell, 1])

1


(array([4.62738619e-08, 0.00000000e+00, 4.62738619e-08]),
 0.039748944692651375,
 False,
 {'amount': 1,
  'curr_step': 3,
  'units_bought': 0,
  'units_sold': 126,
  'balance': 99.37236173162843,
  'net_worth': 99.37236173162843,
  'units_held': 0})